In [ ]:
from PIL import Image
import torch
from transformers import BlipProcessor
from transformers import BlipForQuestionAnswering
def predict_vqa(image_path: str,
                question: str,
                model: BlipForQuestionAnswering,
                processor: BlipProcessor,
                device: torch.device) -> str:
    """
    Prédit la réponse à une question sur une image avec un modèle BLIP-VQA.

    Args:
        image_path (str): chemin vers le fichier image.
        question   (str): texte de la question.
        model      (BlipForQuestionAnswering): modèle finetuné.
        processor  (BlipProcessor): processor associé.
        device     (torch.device): 'cuda' ou 'cpu'.

    Returns:
        str: réponse générée par le modèle.
    """
    # Mode évaluation
    # model.eval()
    # Chargement et prétraitement
    image = Image.open(image_path).convert("RGB")
    inputs = processor(image,
                       question,
                       return_tensors="pt").to(device)

    # Génération
    with torch.no_grad():
        generated_ids = model.generate(**inputs,
                                       max_length=16,
                                       num_beams=5,
                                       early_stopping=True)
    # Décodage
    answer = processor.batch_decode(generated_ids,
                                    skip_special_tokens=True)[0]
    return answer


In [ ]:
processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
mm=torch.load("/content/drive/MyDrive/vqa_model.pt",weights_only=False)

In [ ]:
img_path = "/content/apple.png"
mm.to(device)
question = "What is the name of the object in the image?"
response = predict_vqa(img_path, question, mm, processor, device)
print("Réponse :", response)

Réponse : apple


In [ ]:
img_path = "/content/rr.jpg"
mm.to(device)
question = "describe the content of the image"
response = predict_vqa(img_path, question, mm, processor, device)
print("Réponse :", response)

Réponse : restaurant scene
